## Model Description

NyayaLM v0.5 is a fine-tuned version of Google's Gemma 3n model, specifically trained on Nepali legal documents and question-answer pairs. This model is designed to provide accurate legal information in Nepali, running entirely offline.

## Model Details

- **Base Model**: Google Gemma 3n 4B
- **Fine-tuning Dataset**: Nepali Legal QA (10,980 examples)
- **Languages**: Nepali
- **Domain**: Legal (Nepalese Law)
- **Context Length**: 2048 tokens


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    dtype = None, # None for auto detection
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Let's finetune Gemma 3N!

You can finetune the vision and text parts for now through selection - the audio part can also be finetuned - we're working to make it selectable as well!

We now add LoRA adapters so we only need to update a small amount of parameters!

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 4,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

We get the first 3000 rows of the dataset

In [6]:
from datasets import load_dataset

# Load the dataset with only the columns you need and limit to 10,000 samples
dataset = load_dataset("chhatramani/Nepali_Legal_QA", split="train")  # First 1000 samples
dataset = dataset.select_columns(["instruction", "output"])  # Only keep question and answer columns

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [7]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [8]:
# Load Gemma‑3 chat template
tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

Let's see how row 100 looks like!

In [9]:
dataset[100]

{'instruction': 'यस ऐन अन्तर्गत उजुरी दर्ता गर्नको लागि सीमित अवधि हुनुको उद्देश्य के हो?',
 'output': 'यस ऐन अन्तर्गत उजुरीहरू दायर गर्नको लागि सीमित अवधि हुनुको उद्देश्य भनेको समयमै उजुरीहरू दायर भएको सुनिश्चित गर्नु र लामो अवधि पछि बासी दाबीहरू उठ्नबाट रोक्नु हो।'}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [10]:
# Formatting function wraps each instruction+output into a single-turn Gemma conversation
def formatting_prompts_func(examples):
    convos = []
    for inst, out in zip(examples["instruction"], examples["output"]):
        convos.append([
            {"role": "user",  "content": [{"type": "text", "text": inst}]},
            {"role": "model", "content": [{"type": "text", "text": out}]},
        ])
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        .removeprefix("<bos>")
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [11]:
dataset[100]["text"]

'<start_of_turn>user\nयस ऐन अन्तर्गत उजुरी दर्ता गर्नको लागि सीमित अवधि हुनुको उद्देश्य के हो?<end_of_turn>\n<start_of_turn>model\nयस ऐन अन्तर्गत उजुरीहरू दायर गर्नको लागि सीमित अवधि हुनुको उद्देश्य भनेको समयमै उजुरीहरू दायर भएको सुनिश्चित गर्नु र लामो अवधि पछि बासी दाबीहरू उठ्नबाट रोक्नु हो।<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

Best HyperParameter to Train

In [13]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,     # Use full dataset
    eval_dataset = None,         # No evaluation
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,            # Effective batch = 16
        warmup_steps = 100,
        max_steps = int(len(dataset) * 2 / 16),      # ~2 epochs
        learning_rate = 1e-5,
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        fp16 = True,
        seed = 1234,
        report_to = "none",
        save_steps = 200,
        save_total_limit = 2,
        logging_dir = "logs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10980 [00:00<?, ? examples/s]

Use we below parameter for fast traning as we have only few hours

In [20]:
from trl import SFTTrainer, SFTConfig

# Optimized config for faster training (~2 hours)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,     # Use full dataset
    eval_dataset = None,         # No evaluation
    args = SFTConfig(
        per_device_train_batch_size = 4,              # Faster per-step
        gradient_accumulation_steps = 4,              # Effective batch = 16
        warmup_steps = 50,                            # Shorter warmup
        max_steps = int(len(dataset) / 16),           # 1 epoch
        learning_rate = 2e-5,                         # Slightly more aggressive LR
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        fp16 = True,                                 # Disable fp16
        bf16 = False,                                  # Enable bf16 (faster & stable on newer GPUs)
        seed = 1234,
        report_to = "none",
        save_steps = 200,
        save_total_limit = 2,
        logging_dir = "logs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10980 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [21]:
# We no longer need this, as we've manually created the labels
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/10980 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [22]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nयस ऐन अन्तर्गत उजुरी दर्ता गर्नको लागि सीमित अवधि हुनुको उद्देश्य के हो?<end_of_turn>\n<start_of_turn>model\nयस ऐन अन्तर्गत उजुरीहरू दायर गर्नको लागि सीमित अवधि हुनुको उद्देश्य भनेको समयमै उजुरीहरू दायर भएको सुनिश्चित गर्नु र लामो अवधि पछि बासी दाबीहरू उठ्नबाट रोक्नु हो।<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [23]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                               यस ऐन अन्तर्गत उजुरीहरू दायर गर्नको लागि सीमित अवधि हुनुको उद्देश्य भनेको समयमै उजुरीहरू दायर भएको सुनिश्चित गर्नु र लामो अवधि पछि बासी दाबीहरू उठ्नबाट रोक्नु हो।<end_of_turn>\n'

In [24]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.988 GB of memory reserved.


# Let's train the model!

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,980 | Num Epochs = 1 | Total steps = 686
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 5,283,840 of 5,444,722,112 (0.10% trained)


Step,Training Loss
50,3.082800
100,2.754900
150,2.400700
200,2.202000
250,2.107000
300,2.094200
350,2.046600
400,2.013800
450,1.965400
500,1.979800


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3718.7403 seconds used for training.
61.98 minutes used for training.
Peak reserved memory = 12.592 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 85.422 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>


We can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What are the potential risks and concerns associated with carbon dioxide pneumoperitoneum and pneumothorax in neonatal video-surgery?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The potential risks and concerns associated with carbon dioxide pneumoperitoneum and pneumothorax in neonatal video-surgery include:

* **Respiratory distress:** Neonates have immature respiratory systems and may experience respiratory distress due to the introduction of carbon dioxide into the abdominal cavity.
* **Increased intra-abdominal pressure:** Carbon dioxide pneumoperitoneum can increase intra-abdominal pressure, potentially leading to complications such as bowel distension or perforation.
* **Pneumothorax:** There is a risk of pneumothorax occurring during video-surgery, especially if the carbon dioxide is introduced rapidly or if there is an underlying lung disease.
* **Lung injury:** Carbon dioxide pneumoperitoneum can potentially cause lung injury, especially in neonates with fragile lungs.
* **Gas embolism:** There is a rare risk of gas embolism occurring during video-surgery if the carbon dioxide is introduced too rapidly or if there is a vascular perforation.

These ri

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("medqa-gemma-3n")  # Local saving
tokenizer.save_pretrained("medqa-gemma-3n")

In [ ]:
model.push_to_hub("chhatramani/medqa-gemma-3nE4B-4bit", token = "HF_TOKEN") # Online saving
tokenizer.push_to_hub("chhatramani/medqa-gemma-3nE4B-4bit", token = "HF_TOKEN") # Online saving

README.md:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/76.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/chhatramani/medqa-gemma-3nE4B-4bit


README.md:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chhatramani/medqa-gemma-3nE4B-4bit/commit/d8ef05c49acc867b5d65eb0bbf48664c8545a786', commit_message='Upload tokenizer', commit_description='', oid='d8ef05c49acc867b5d65eb0bbf48664c8545a786', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chhatramani/medqa-gemma-3nE4B-4bit', endpoint='https://huggingface.co', repo_type='model', repo_id='chhatramani/medqa-gemma-3nE4B-4bit'), pr_revision=None, pr_num=None)

Evaluation

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3N-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3N-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3N-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-3N-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3N-finetune.gguf` file or `gemma-3N-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
